Example below shows how we fit resourse estimation for MultiplyAdd operation to 2-nd degree
polynomial of 2 variables - n (register size) and radix.

In [4]:
from psiqworkbench import QPU, QFixed, resource_estimator

from qmath.utils.re_utils import FILTERS_FOR_NUMERIC_RE
from qmath.func.common import MultiplyAdd
import numpy as np

small_to_zero = lambda a: np.array([0 if abs(x)<1e-9 else x for x in a])

def re_numeric(n, r, r3):
    qpu = QPU(filters=FILTERS_FOR_NUMERIC_RE)
    qpu.reset(10 * n)
    dst = QFixed(n, name="dst", radix=r, qpu=qpu)
    lhs = QFixed(n, name="lhs", radix=r, qpu=qpu)
    rhs = QFixed(n, name="rhs", radix=r3, qpu=qpu)

    MultiplyAdd().compute(dst, lhs, rhs)

    re = resource_estimator(qpu)
    return re.resources()

assgns = []
x = []
res = []
for n in range(20,30):
    for diff1 in [-2, -1, 0, 1, 2]:
        r = n//2+diff1
        for diff3 in [-2, -1, 0, 1, 2]:
            r3 = r+diff3
            res.append(re_numeric(n,r,r3))
            x.append([1, n, r, r3, n**2, r**2, r3**2, n*r, n*r3, r*r3])
x = np.array(x)

for metric in ["gidney_lelbows", "gidney_relbows", "toffs","qubit_highwater", "active_volume"]:
    y = np.array([re[metric] for re in res])
    coeffs, residuals, rank, s = np.linalg.lstsq(x, y, rcond=None)
    coeffs=small_to_zero(coeffs)
    print(metric, coeffs)

gidney_lelbows [-16.    7.5   0.   -0.5   0.5   0.    0.5   0.    1.    0. ]
gidney_relbows [-16.    7.5   0.   -0.5   0.5   0.    0.5   0.    1.    0. ]
toffs [-8.   8.5  0.   0.5  0.5  0.   0.5  0.   1.   0. ]
qubit_highwater [1. 4. 0. 2. 0. 0. 0. 0. 0. 0.]
active_volume [-1.16534785e+03  8.20706681e+02  2.31160389e-01 -2.30000000e+01
  5.90023022e+01  1.06063720e-02  5.40057143e+01 -4.60431655e-03
  1.18000000e+02 -1.14285714e-02]
